<a href="https://colab.research.google.com/github/lukehdez95/DS-Unit-2-Linear-Models/blob/master/LS_DS_212_Assignment_LH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# Checking Data

In [4]:
#I first want to make sure my data only includes the data from the years and months that I want to include
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# I see that the date created is a string value, in order to verify that all my data is within 2016, I want to extract the year as a different feature so that I may compare the year of each observation
def created_yr(TIMEDATE):
  return int(TIMEDATE.split('-')[0])
#this function should take the time date string and return the year, I will check that it works properly below
created_yr('2016-06-24 07:54:24')

2016

In [6]:
#I will need a similar function for the month later
def created_mth(TIMEDATE):
  return int(TIMEDATE.split('-')[1])
created_mth('2016-06-24 07:54:24')

6

In [7]:
#Now I can use my functions to create two new features that have the solely the year information and the month information respectively
df['created_yr'] = df['created'].apply(created_yr)
df['created_mth'] = df['created'].apply(created_mth)
print(df['created_yr'].unique())
print(df['created_mth'].unique())

[2016]
[6 4 5]


In [8]:
# I can now confirm that all my data is from 2016, and the months of 4, 5, or 6

# Making 2 New Features

In [9]:
# The 2 new features I want to make are the total number of rooms and the bed to bath ratio
df['rooms_TOTAL'] = df['bathrooms'] + df['bedrooms']
df['bed_to_bath'] = df['bedrooms']/df['bathrooms']
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,created_yr,created_mth,rooms_TOTAL,bed_to_bath
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,6,4.5,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,6,3.0,2.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2.0,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2.0,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,5.0,4.0


In [14]:
# If any units exist with no bathrooms, it would lead to a ratio of inf or NaN
df[df['bathrooms']==0]

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,created_yr,created_mth,rooms_TOTAL,bed_to_bath
240,0.0,0,2016-06-12 14:12:49,,46th Street,40.6503,-74.0135,2195,250 46th Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,6,0.0,NaN
292,0.0,1,2016-04-20 02:46:16,Sky high views with extra high ceiling! This c...,E 29th St.,40.7450,-73.9863,4475,10 E 29th St.,low,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,1.0,inf
300,0.0,0,2016-04-07 02:49:51,"Enjoy the great restaurants, clubs, boutiques ...",408 East 92nd Street,40.7805,-73.9464,2775,408 East 92nd Street,low,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,2016,4,0.0,NaN
455,0.0,1,2016-04-16 03:45:50,"Upper West Side-1 BER- Doorman, Elevat...",West 70th Street,40.7775,-73.9835,3950,210 West 70th Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,1.0,inf
481,0.0,0,2016-04-05 02:49:40,Amidst the vibrant energy of the West Village ...,10 Downing Street,40.7295,-74.0029,3695,10 Downing Street,low,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2016,4,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47619,0.0,1,2016-04-10 03:28:40,IPN is a development of three towers w...,Harrison St,40.7191,-74.0109,4595,40 Harrison St,low,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,1.0,inf
48557,0.0,1,2016-04-22 05:01:23,"Please contact, If interested<br />Or,...",John Street,40.7084,-74.0066,3650,85 John Street,low,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,1.0,inf
48834,0.0,0,2016-04-14 04:09:51,Studio on Upper West Side,W 87th St,40.7894,-73.9764,2300,265 W 87th St,low,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,0.0,NaN
49091,0.0,1,2016-04-06 17:58:16,,E 34th St,40.7454,-73.9776,3725,222 E 34th St,low,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,1.0,inf


In [16]:
# I am going to make a new df, including only the units that have bathrooms/bedrooms, which would mean I am removing 304 observations, out of my approximately 50000.
bathdf = df[df['bathrooms'] != 0]
bathdf

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,created_yr,created_mth,rooms_TOTAL,bed_to_bath
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,6,4.5,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,6,3.0,2.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2.0,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2.0,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,2,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,40.7426,-73.9790,3200,230 E 30 St,medium,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,6,3.0,2.0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2016,4,2.0,1.0
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2.0,1.0
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,1.0,0.0


# Train/Test Split

In [17]:
#Splitting data into train and test
train = bathdf[bathdf['created_mth'] <= 5]
test = bathdf[bathdf['created_mth'] == 6]
train.shape, test.shape

((31653, 38), (16860, 38))

In [18]:
# Ensuring that the right months were split into the right dataframe
print(train['created_mth'].unique())
print(test['created_mth'].unique())

[4 5]
[6]


# Multiple Linear Regression

In [19]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
#importing all the libraries i need access to

In [20]:
# instantiating and arranging features matrices and target vectors
target = 'price'
features = ['rooms_TOTAL','bed_to_bath']
y_train = train[target]
y_test = test[target]
X_train = train[features]
X_test = test[features]
model = LinearRegression()

In [21]:
model.fit(X_train,y_train)
#fit the model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [26]:
#getting and interpreting model intercepts and coefficients
model.intercept_,model.coef_

(1435.2352506106713, array([1135.25574555, -772.12347113]))

This means that an apartment with no rooms and a bed to bath ratio of 0, would cost 1435 dollars (our intercept) to rent.

Since our first coefficient is 1135, it means that for every additional room(bedroom OR bathroom) in a unit, the price goes up by $1135. (our first coefficient was the feature of total rooms in the unit)

Since our second coefficient is -772, it means that the higher the bed to bath ratio is, the less the price to rent the unit is.(since our 2nd coefficient was the feature of bed to bath ratio)


# Regression Metrics

In [29]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
#importing regression metrics

In [27]:
#MAE
print('Training MAE is:', mean_absolute_error(y_train, model.predict(X_train)))
print('Test MAE is:', mean_absolute_error(y_test, model.predict(X_test)))

Training MAE is: 821.691005046723
Test MAE is: 828.3422664808853


In [30]:
#RMSE
print('Training RMSE is:',mean_squared_error(y_train,model.predict(X_train), squared=False))
print(' Test RMSE is:', mean_squared_error(y_test,model.predict(X_test), squared=False))

Training RMSE is: 1233.457852495355
 Test RMSE is: 1225.2462282462627


In [31]:
#R2
print('Training R2 is:',model.score(X_train,y_train))
print('Test R2 is:', model.score(X_test,y_test))

Training R2 is: 0.5089352609102014
Test R2 is: 0.5171154712743637
